In [ ]:
from yahoo_fin.stock_info import get_data
import pandas as pd
import numpy as np

In [ ]:
latest_25_candles = get_data('EMR', index_as_date=False).tail(25)
latest_25_candles

In [ ]:
# Drop the adjusted close column
latest_25_candles = latest_25_candles.drop(['adjclose'], axis=1)

In [ ]:
# Get the selling pressure (i.e. distance between candle's high and close)
latest_25_candles['SellingPressure'] = latest_25_candles['high'] - latest_25_candles['close']

# Get the length of candle's body (from open to close)

latest_25_candles['O-to-C'] = latest_25_candles['close'] - latest_25_candles['open']

# Get the rolling mean of the candles' bodies for recent 20 candles

latest_25_candles['OC-20D-Mean'] = latest_25_candles['O-to-C'].rolling(20).mean()

# Get the % change of the current OC relative from the rolling mean

latest_25_candles['OC-%-from-20D-Mean'] = 100*(latest_25_candles['O-to-C'] - latest_25_candles['OC-20D-Mean'])/latest_25_candles['OC-20D-Mean']

# Get the maximum OC compared to the recent 10 candles

latest_25_candles['MaxOC_Prev10'] = latest_25_candles['O-to-C'].rolling(10).max()

# Get the rolling mean of volume for the recent 20 candles

latest_25_candles['Volume-20D-Mean'] = latest_25_candles['volume'].rolling(20).mean()

# Get the % change of the current volume relative from the rolling mean

latest_25_candles['Volume-%-from-20D-Mean'] = 100*(latest_25_candles['volume'] - latest_25_candles['Volume-20D-Mean'])/latest_25_candles['Volume-20D-Mean']

latest_25_candles

In [ ]:
latest_5_candles = latest_25_candles.tail(5)
latest_5_candles

In [ ]:
condition = (latest_5_candles['O-to-C'] >= 0.0) & (latest_5_candles['O-to-C'] == latest_5_candles['MaxOC_Prev10']) & (latest_5_candles['SellingPressure']/latest_5_candles['O-to-C'] <= 0.40) & (latest_5_candles['OC-%-from-20D-Mean'] >= 100.0) & (latest_5_candles['Volume-%-from-20D-Mean'] >= 50.0)

breakouts = latest_5_candles[condition].reset_index(drop=True)

breakouts